In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from bs4 import BeautifulSoup
import urllib
import numpy as np
import re
import pandas as pd
import math

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [32]:
caps = DesiredCapabilities().CHROME
#caps["pageLoadStrategy"] = "normal"  #  complete
caps["pageLoadStrategy"] = "eager"  #  interactive
#caps["pageLoadStrategy"] = "none

# Loading up the Glassdoor main page
s = Service('/Users/safadjerir/Desktop/projetDatavis/chromedriver')
driver = webdriver.Chrome(service=s)

driver.get("https://www.glassdoor.fr/Emploi/france-data-analyst-emplois-SRCH_IL.0,6_IN86_KO7,21_IP{}.htm".format(1))
    

time.sleep(1)

source = driver.page_source
soup = BeautifulSoup(source)

page_list = soup.select("#MainCol > div:nth-child(2) > div")[0].text
page = int(list(map(float, re.findall(r"[-+]?\d*\.\d+|\d+", page_list)))[1])





In [33]:
liste_note = []
liste_ent = []
liste_int = []
liste_ville = []
liste_salaire = []
liste_variance_salaire = []
liste_periode_public = []
liste_top = []
liste_description = []

## "https://www.glassdoor.fr/Emploi/france-data-analyst-emplois-SRCH_IL.0,6_IN86_KO7,21_IP{}

for i in range(1,11):
    ## Parcours de toutes les pages
    driver.get("https://www.glassdoor.fr/Emploi/france-data-analyst-emplois-SRCH_IL.0,6_IN86_KO7,21_IP{}.htm".format(i))
    source = driver.page_source
    soup = BeautifulSoup(source)
    time.sleep(1)
    
    for el in soup.select("#MainCol > div:nth-child(1) > ul > li "):
        ##On reste bien sur la bonne page
        #driver.get("https://www.glassdoor.fr/Emploi/france-data-scientist-emplois-SRCH_IL.0,6_IN86_KO7,21_IP{}.htm".format(i))
    
        ## Liste des notes
        try:
            a = float(el.select("li > div > span")[0].text.replace(",", "."))
        except IndexError as error:
            a = np.nan
        liste_note.append(a)

        ## Liste entreprise
        try:
            b = el.select('li > div:nth-child(2) > div > a > span')[0].text
        except:
            b = "Pas d'entreprise"
        liste_ent.append(b)

        ## Intitulé poste
        try:
            c= el.select('li > div:nth-child(2) > a > span')[0].text
        except:
            c= "Erreur"
        liste_int.append(c)

        ## Ville
        try:
            d = el.select('li > div:nth-child(2) > div')[1].select("div> span")[0].text
        except:
            d= "inconnue"
        liste_ville.append(d)

        ## Salaire 
        try:
            e0= el.select('li > div:nth-child(2) > div')[2].select("div > span")[0].text.replace(u'\xa0', u' ')
            try:
                e_list = list(map(float, re.findall(r"[-+]?\d*\.\d+|\d+", e0)))
                e = e_list[0]
                v = e_list[-1]
            except:
                e= np.nan
                v= np.nan
        except IndexError as error:
            e= np.nan
            v=np.nan
        liste_salaire.append(e)
        liste_variance_salaire.append(v)

        ## Jours
        if math.isnan(e):
            try:
                f= el.select('li > div:nth-child(2)')[0].select('div')[-1].text
            except: 
                f= 'inconnu'
        else:
            try:
                f = el.select('li > div:nth-child(2)')[0].select('div')[6].select('div')[-1].text
            except:
                f = 'inconnu'
        liste_periode_public.append(f)

        ## Top société
        if "Top société" in str(el):
            liste_top.append(1)
        else:
            liste_top.append(0)
            
        
        ## Descrp
        extension = el.find_all('a')[0]['href']
        href = "https://www.glassdoor.fr" + extension
        
        
        driver_sub_dep = webdriver.Chrome('/Users/safadjerir/Desktop/projetDatavis/chromedriver')
        driver_sub_dep.get(href)
        time.sleep(1)
        source_2 = driver_sub_dep.page_source
        soup_2 = BeautifulSoup(source_2)
        
        
        descr = soup_2.select("#JobDescriptionContainer")[0].text.replace('\n', ' ')
        
        
        liste_description.append(descr)
        
        driver_sub_dep.quit()
        
        
        
driver.quit()

<ipython-input-33-8641f7b49b52>:93: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_sub_dep = webdriver.Chrome('/Users/safadjerir/Desktop/projetDatavis/chromedriver')


In [35]:
df2 = pd.DataFrame(list(zip(liste_ent, liste_note, liste_int, liste_ville, liste_periode_public,
                          liste_salaire, liste_variance_salaire, liste_top, liste_description)), 
                  columns=['entreprise', 'note', 'poste', 'ville', 'publication', 'salaire_bas', 
                           'salaire_haut', 'top_societe', 'descr'])

In [36]:
df2

,entreprise,note,poste,ville,publication,salaire_bas,salaire_haut,top_societe,descr
0,Groupama Nord Est,3.8,Data Analyst / Data Scientist H/F,Reims,+ de 30 j,30.0,53.0,0,Des ambitions fortes et partagées résumées par...
1,L'Oreal,4.2,Data Quality Analyst Apprentice,Saint-Ouen,24h,52.0,61.0,1,WANT TO KNOW MORE ABOUT OUR BIG TECH TRANSFORM...
2,Carrefour,3.7,ALTERNANCE - Data Analyst (H/F),Évry,+ de 30 j,NaN,NaN,0,Créateur de l’hypermarché et leader de la tran...
3,MAIF,4.0,DATA ANALYST - NIORT H/F,Niort,24h,NaN,NaN,0,Intitulé du poste DATA ANALYST - NIORT H/F Ty...
4,Harmonie Mutuelle,3.4,Data Analyst H/F,Paris,3 j,45.0,55.0,0,Nous recherchons : 1 Data Analyst H/F Harmon...
...,...,...,...,...,...,...,...,...,...
295,TotalEnergies,4.0,STAGE - Data analyst H/F,Paris,3 j,NaN,NaN,0,Systèmes d'Information PARIS-RUE DES JEÛNEURS(...
296,KPMG,3.9,CONSULTANT DÉBUTANT DATA ANALYST - CONNECTED T...,Courbevoie,24h,36.0,54.0,0,Pourquoi devenir Consultant Débutant Data Anal...
297,SOCIO DATA MANAGEMENT,5.0,Consultant(e) Data Analyst – Confirmé(e),Paris,+ de 30 j,36.0,54.0,0,"Contexte Société à taille humaine, Socio Data..."
298,EDF Energies Nouvelles,4.0,STAGE DATA ANALYST H/F,Paris,25 j,35.0,62.0,0,"Filiale du groupe EDF, EDF Renouvelables est u..."


In [37]:
df3 = df2.copy()

In [39]:
df3.drop_duplicates()

,entreprise,note,poste,ville,publication,salaire_bas,salaire_haut,top_societe,descr
0,Groupama Nord Est,3.8,Data Analyst / Data Scientist H/F,Reims,+ de 30 j,30.0,53.0,0,Des ambitions fortes et partagées résumées par...
1,L'Oreal,4.2,Data Quality Analyst Apprentice,Saint-Ouen,24h,52.0,61.0,1,WANT TO KNOW MORE ABOUT OUR BIG TECH TRANSFORM...
2,Carrefour,3.7,ALTERNANCE - Data Analyst (H/F),Évry,+ de 30 j,NaN,NaN,0,Créateur de l’hypermarché et leader de la tran...
3,MAIF,4.0,DATA ANALYST - NIORT H/F,Niort,24h,NaN,NaN,0,Intitulé du poste DATA ANALYST - NIORT H/F Ty...
4,Harmonie Mutuelle,3.4,Data Analyst H/F,Paris,3 j,45.0,55.0,0,Nous recherchons : 1 Data Analyst H/F Harmon...
5,Softeam,3.8,Business Analyst Data,Paris,+ de 30 j,60.0,70.0,0,Diffusée par: Yasmina Hamimi Responsable Recru...
6,Groupe Les Mousquetaires,3.5,Data Analyst H/F (IT),Bondoufle,+ de 30 j,40.0,45.0,0,Le Groupement Les Mousquetaires a créé en 2021...
7,France Télévisions,NaN,Alternant(e) Assistant(e) Data Analyst H/F,Paris,24h,NaN,NaN,0,Informations généralesDirection/ServiceGestion...
8,Winamax,4.2,Data Analyst Junior (H/F),Paris,7 j,NaN,NaN,0,Pour renforcer la capacité de Winamax à exploi...
9,HOLOMNIS,NaN,Chargé(e) d’études – Data Analyst – Data Scien...,Paris,27 j,38.0,38.0,0,"Holomnis, Cabinet de Conseil en Ressources Hum..."


In [40]:
df3.to_excel("dataAnalyst_jobs2.xlsx")

In [41]:
df3.drop_duplicates().to_excel("dataAnalyst_jobs.xlsx_no_duplicates2.xlsx")

In [42]:
df_ancien = pd.read_excel ('dataAnalyst_jobs_no_duplicates.xlsx')

In [43]:
df_ancien

,Unnamed: 0,entreprise,note,poste,ville,publication,salaire_bas,salaire_haut,top_societe,descr
0,0,BNP Paribas,3.8,DATA ANALYST - PYTHON H/F,Issy-les-Moulineaux,+ de 30 j,NaN,NaN,0,DATA ANALYST - PYTHON H/F (NUMÉRO DE L'EMPLOI ...
1,1,UNLCK,5.0,Data Analyst – Agrégateur Datatech - H/F,Paris,17 j,45.0,45.0,0,Postgresql / Tableau / Python / Sql / Mysql Bi...
2,2,PRESTINFO SERVICES,NaN,Data Analyst F/H,Lyon,5 j,45.0,45.0,0,Nous recherchons pour notre client basé à Lyon...
3,3,Groupe Les Mousquetaires,3.3,Data Analyst H/F (IT),Bondoufle,19 j,40.0,45.0,0,Le Groupement Les Mousquetaires a créé en 2021...
4,4,HOLOMNIS,NaN,Chargé(e) d’études – Data Analyst – Data Scien...,Paris,7 j,38.0,38.0,0,"Holomnis, Cabinet de Conseil en Ressources Hum..."
5,5,Allianz France,4.0,Chargé(e) d'études statistiques - Data Analyst...,Courbevoie,11 j,NaN,NaN,0,Description de la mission Dans un contexte de...
6,6,Winamax,4.2,Data Analyst Junior (H/F),Paris,7 j,NaN,NaN,0,Pour renforcer la capacité de Winamax à exploi...
7,7,Ysance,3.6,Data Analyst,Levallois-Perret,1 j,NaN,NaN,0,Description de l'entreprise*Qui sommes-nous ?*...
8,8,Le Bon Marché,4.0,Data Analyst CDI F/H,Paris,+ de 30 j,30.0,45.0,0,Au sein de la Direction Marketing Client et ra...
9,9,CHRONOFRESH,3.0,Data Analyst H/F,Paris,4 j,35.0,40.0,0,"Filiale de Chronopost, nous sommes LE spéciali..."


In [44]:
dfconcat = pd.concat([df3.drop_duplicates(),df_ancien])

In [45]:
dfconcat

,entreprise,note,poste,ville,publication,salaire_bas,salaire_haut,top_societe,descr,Unnamed: 0
0,Groupama Nord Est,3.8,Data Analyst / Data Scientist H/F,Reims,+ de 30 j,30.0,53.0,0,Des ambitions fortes et partagées résumées par...,NaN
1,L'Oreal,4.2,Data Quality Analyst Apprentice,Saint-Ouen,24h,52.0,61.0,1,WANT TO KNOW MORE ABOUT OUR BIG TECH TRANSFORM...,NaN
2,Carrefour,3.7,ALTERNANCE - Data Analyst (H/F),Évry,+ de 30 j,NaN,NaN,0,Créateur de l’hypermarché et leader de la tran...,NaN
3,MAIF,4.0,DATA ANALYST - NIORT H/F,Niort,24h,NaN,NaN,0,Intitulé du poste DATA ANALYST - NIORT H/F Ty...,NaN
4,Harmonie Mutuelle,3.4,Data Analyst H/F,Paris,3 j,45.0,55.0,0,Nous recherchons : 1 Data Analyst H/F Harmon...,NaN
...,...,...,...,...,...,...,...,...,...,...
37,Oh BiBi,4.6,Data analyst,Paris,27 j,NaN,NaN,0,Oh BiBi recrute un(e) Data Analyst. Au sein de...,114.0
38,Lunii,4.5,Data Analyst F/H (CDI),Paris,27 j,28.0,43.0,0,Vous rejoindrez notre Pôle Marketing et vous f...,115.0
39,Marionnaud,3.0,Business Analyst CDI H/F,Paris,20 j,NaN,NaN,0,BUSINESS ANALYST CDI H/F Rattaché(e) au Busine...,116.0
40,PwC,3.9,Data analyst | Deals (M&A) | CDI | H/F,Neuilly,26 j,NaN,NaN,0,Line of Service Advisory Industry/Sector Not A...,145.0


In [49]:
dfconcat.to_excel("dataAnalyst_jobs_final.xlsx")

In [46]:
dff = dfconcat.copy()


In [47]:
len(dff.drop_duplicates())

89

In [48]:
dff.drop_duplicates(subset=dff.columns.difference(['publication']))

,entreprise,note,poste,ville,publication,salaire_bas,salaire_haut,top_societe,descr,Unnamed: 0
0,Groupama Nord Est,3.8,Data Analyst / Data Scientist H/F,Reims,+ de 30 j,30.0,53.0,0,Des ambitions fortes et partagées résumées par...,NaN
1,L'Oreal,4.2,Data Quality Analyst Apprentice,Saint-Ouen,24h,52.0,61.0,1,WANT TO KNOW MORE ABOUT OUR BIG TECH TRANSFORM...,NaN
2,Carrefour,3.7,ALTERNANCE - Data Analyst (H/F),Évry,+ de 30 j,NaN,NaN,0,Créateur de l’hypermarché et leader de la tran...,NaN
3,MAIF,4.0,DATA ANALYST - NIORT H/F,Niort,24h,NaN,NaN,0,Intitulé du poste DATA ANALYST - NIORT H/F Ty...,NaN
4,Harmonie Mutuelle,3.4,Data Analyst H/F,Paris,3 j,45.0,55.0,0,Nous recherchons : 1 Data Analyst H/F Harmon...,NaN
...,...,...,...,...,...,...,...,...,...,...
37,Oh BiBi,4.6,Data analyst,Paris,27 j,NaN,NaN,0,Oh BiBi recrute un(e) Data Analyst. Au sein de...,114.0
38,Lunii,4.5,Data Analyst F/H (CDI),Paris,27 j,28.0,43.0,0,Vous rejoindrez notre Pôle Marketing et vous f...,115.0
39,Marionnaud,3.0,Business Analyst CDI H/F,Paris,20 j,NaN,NaN,0,BUSINESS ANALYST CDI H/F Rattaché(e) au Busine...,116.0
40,PwC,3.9,Data analyst | Deals (M&A) | CDI | H/F,Neuilly,26 j,NaN,NaN,0,Line of Service Advisory Industry/Sector Not A...,145.0


In [50]:
dff = dff.drop('Unnamed: 0', 1)
dff

,entreprise,note,poste,ville,publication,salaire_bas,salaire_haut,top_societe,descr
0,Groupama Nord Est,3.8,Data Analyst / Data Scientist H/F,Reims,+ de 30 j,30.0,53.0,0,Des ambitions fortes et partagées résumées par...
1,L'Oreal,4.2,Data Quality Analyst Apprentice,Saint-Ouen,24h,52.0,61.0,1,WANT TO KNOW MORE ABOUT OUR BIG TECH TRANSFORM...
2,Carrefour,3.7,ALTERNANCE - Data Analyst (H/F),Évry,+ de 30 j,NaN,NaN,0,Créateur de l’hypermarché et leader de la tran...
3,MAIF,4.0,DATA ANALYST - NIORT H/F,Niort,24h,NaN,NaN,0,Intitulé du poste DATA ANALYST - NIORT H/F Ty...
4,Harmonie Mutuelle,3.4,Data Analyst H/F,Paris,3 j,45.0,55.0,0,Nous recherchons : 1 Data Analyst H/F Harmon...
...,...,...,...,...,...,...,...,...,...
37,Oh BiBi,4.6,Data analyst,Paris,27 j,NaN,NaN,0,Oh BiBi recrute un(e) Data Analyst. Au sein de...
38,Lunii,4.5,Data Analyst F/H (CDI),Paris,27 j,28.0,43.0,0,Vous rejoindrez notre Pôle Marketing et vous f...
39,Marionnaud,3.0,Business Analyst CDI H/F,Paris,20 j,NaN,NaN,0,BUSINESS ANALYST CDI H/F Rattaché(e) au Busine...
40,PwC,3.9,Data analyst | Deals (M&A) | CDI | H/F,Neuilly,26 j,NaN,NaN,0,Line of Service Advisory Industry/Sector Not A...


In [51]:
len(dff.drop_duplicates())

87

In [53]:
mdf = dff.drop_duplicates(subset=dff.columns.difference(['publication']))

In [54]:
mdf.to_excel("dataAnalyst_jobs_final.xlsx")

In [4]:
df_ancien1 = pd.read_excel ('dataScientist_jobs_no_duplicates.xlsx')
df_ancien2 = pd.read_excel ('dataScientist_jobs_no_duplicates2.xlsx')
dfconcat2 = pd.concat([df_ancien1,df_ancien2])
df1 = dfconcat2.copy()
df1 = df1.drop('Unnamed: 0', 1)
ndf = df1.drop_duplicates(subset=df1.columns.difference(['publication']))
len(ndf)
ndf.to_excel("dataScientist_jobs_final.xlsx")